In [1]:
import pandas as pd
import numpy as np
import json
from typing import List, Dict, Any

In [2]:
# hardcoded from SPM's notebook
index_list = [2432,
 19667,
 21392,
 22585,
 22789,
 25822,
 27428,
 38938,
 39901,
 44402,
 46039,
 49823,
 53391,
 60920,
 63940,
 78318,
 79480,
 82643,
 94483,
 100157,
 101519,
 106367,
 106820,
 110718,
 111440,
 117036,
 124222,
 134264,
 140824,
 146126,
 149506,
 160988,
 173987,
 190304,
 208055,
 217691,
 223678,
 231119,
 239336,
 241520,
 241620,
 248166,
 265165,
 265647,
 283211,
 285732,
 290398,
 295864,
 305128,
 314021,
 5598,
 11928,
 14858,
 29023,
 30451,
 39933,
 41322,
 49714,
 55744,
 65791,
 72081,
 83153,
 85971,
 89405,
 99713,
 104179,
 108523,
 110120,
 113473,
 118469,
 120351,
 124776,
 135202,
 139937,
 140373,
 145813,
 147664,
 159295,
 160258,
 166720,
 171849,
 176142,
 180195,
 184776,
 188565,
 190057,
 197077,
 205221,
 207720,
 214001,
 231948,
 233511,
 238215,
 240196,
 248247,
 251860,
 258084,
 262055,
 279586,
 281063]

In [3]:
physician_validation_path = "/Users/sandychen/Library/CloudStorage/Box-Box/Prospective Labeling V3/Data/Stephen Labels.xlsx"
enhance_path_jsonl = "../DSPy_results_batch_previously_labeled_100_dedup_with_prev_msg_w_ref_colbert/labeler_results.jsonl"
baseline_path_jsonl = "../DSPy_results_batch_previously_labeled_100_dedup_with_prev_msg_w_ref/labeler_results.jsonl"
# enhance_path_jsonl = "/Users/sandychen/Library/CloudStorage/Box-Box/Prospective Labeling V3/Pipeline_v2_data/DSPy_results_batch_previously_labeled_100_dedup_with_prev_msg_w_ref_colbert/labeler_results.jsonl"
# baseline_path_jsonl = "/Users/sandychen/Library/CloudStorage/Box-Box/Prospective Labeling V3/Pipeline_v2_data/DSPy_results_batch_previously_labeled_100_dedup_with_prev_msg_w_ref_colbert/labeler_results.jsonl"
# codebook_path = "../data/input_data/dedup_codebook.csv"
codebook_path = "/Users/sandychen/Library/CloudStorage/Box-Box/Prospective Labeling V3/Pipeline_v2_data/dedup_codebook.csv"

In [4]:
Physician_label = pd.read_excel(physician_validation_path)
Physician_label_100 = Physician_label[Physician_label["Index"].isin(index_list)].sort_values(by="Index")
codebook_df = pd.read_csv(codebook_path)[["Dedup Error Code", "Dedup Subdomain", "Dedup Domain", "Dedup Definition"]]

In [5]:
import json
import pandas as pd

def read_jsonl_robust(file_path):
    """Read JSONL with error reporting"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line_num, line in enumerate(file, 1):
            line = line.strip()
            if line:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error line {line_num}: {e}")
    return data
    

def expand_jsonl_to_error_codes(df):
    """
    Expand JSONL DataFrame to flattened structure with error codes.
    Only keeps subdomains where yes=True and expands error_codes.
    
    Returns DataFrame with columns:
    Index, Domain, Subdomain, Error Code, Rationale, confidence, error_summary, error_highlights, confidence
    """
    expanded_rows = []
    
    for _, row in df.iterrows():
        # Extract basic info
        index_val = row['index']
        domain = row['domain']
        error_summary = row['error_summary']
        error_highlights = row['error_highlights']
        
        # Process subdomains - check if subdomains exists and is not None
        if 'subdomains' in row and row['subdomains'] is not None:
            for subdomain_data in row['subdomains']:
                # Only process subdomains where yes=True
                if subdomain_data.get('yes', False):
                    subdomain_name = subdomain_data.get('subdomain', '')
                    subdomain_rationale = subdomain_data.get('rationale', '')
                    subdomain_confidence = subdomain_data.get('confidence', 0.0)
                    
                    # Process error codes within this subdomain
                    if 'error_codes' in subdomain_data:
                        for error_code_data in subdomain_data['error_codes']:
                            # Only keep error codes where yes=True
                            if error_code_data.get('yes', False):
                                expanded_rows.append({
                                    'Index': index_val,
                                    'Domain': domain,
                                    'Subdomain': subdomain_name,
                                    'Error Code': error_code_data.get('error_code', ''),
                                    'Rationale': error_code_data.get('rationale', ''),
                                    'confidence': error_code_data.get('confidence', 0.0),
                                    'error_summary': error_summary,
                                    'error_highlights': error_highlights,
                                    'subdomain_confidence': subdomain_confidence
                                })
                    else:
                        # If no error_codes but subdomain yes=True, create a row without error code
                        expanded_rows.append({
                            'Index': index_val,
                            'Domain': domain,
                            'Subdomain': subdomain_name,
                            'Error Code': "not labeled by LLM",
                            'Rationale': subdomain_rationale,
                            'confidence': subdomain_confidence,
                            'error_summary': error_summary,
                            'error_highlights': error_highlights,
                            'subdomain_confidence': subdomain_confidence
                        })
    
    return pd.DataFrame(expanded_rows)

In [6]:
baseline_df = pd.DataFrame(read_jsonl_robust(baseline_path_jsonl)).sort_values(by="index")
enhance_df = pd.DataFrame(read_jsonl_robust(enhance_path_jsonl)).sort_values(by="index")
baseline_expanded = expand_jsonl_to_error_codes(baseline_df)
enhance_expanded = expand_jsonl_to_error_codes(enhance_df)


In [7]:
baseline_expanded

,Index,Domain,Subdomain,Error Code,Rationale,confidence,error_summary,error_highlights,subdomain_confidence
0,2432,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,The directive to schedule only via the app wit...,0.68,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.70
1,2432,Clinical Reasoning,Comprehension of Patient Query,Misinterpretation of Clinical Query,Patient requested steps to see a colleague; re...,0.81,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.82
2,2432,Clinical Reasoning,Comprehension of Patient Query,Incomplete Response to Patient Query,It does not explain how to book with the colle...,0.79,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.82
3,2432,Clinical Reasoning,Workflow Recommendations,Omitted Workflow Information,Directs only to the app without offering phone...,0.87,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.86
4,5598,Clinical Reasoning,Comprehension of Patient Query,Incomplete Response to Patient Query,It does not acknowledge/confirm the appointmen...,0.87,The response does not acknowledge or confirm t...,"[{'id': 'H1', 'excerpt': 'I don't have access ...",0.87
...,...,...,...,...,...,...,...,...,...
725,314021,Clinical Reasoning,Triage,Inadequate Risk Stratification,The reply provides no assessment of risk level...,0.72,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.76
726,314021,Clinical Reasoning,Workflow Recommendations,Omitted Workflow Information,No timeframe or next steps for referral placem...,0.78,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.78
727,314021,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,"Provides unclear directions (""place a referral...",0.79,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.79
728,314021,Communication Quality & Readability,Formatting,Formatting / Display Error,Message ends with an incomplete sign-off lacki...,0.66,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.66


In [8]:
Physician_label_100

,Index,Source,Domain,Subdomain,Error Code,Rationale,Free Text Comments
0,2432,Human,Clinical Reasoning,Workflow Recommendations,Violation of Standard Workflows,Clinic appears to schedule directly via messag...,NaN
1,5598,Baseline,Clinical Reasoning,Comprehension of Patient Query,Off-topic or Irrelevant Reply to Patient Query,The response is irrelevant to the patient's or...,NaN
2,5598,Enhanced,Clinical Reasoning,Comprehension of Patient Query,Off-topic or Irrelevant Reply to Patient Query,The response gives unrelated information about...,NaN
3,11928,Human,Clinical Reasoning,Comprehension of Patient Context,Incorrect Clinical Information,LLM did not realize that the CBC had already b...,NaN
4,11928,Enhanced,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,The instructions are contradictory since they ...,NaN
...,...,...,...,...,...,...,...
157,285732,Human,Clinical Reasoning,Assessment,Omitted Verification of Incomplete Patient Inf...,The LLM made a resopnse despite not being able...,NaN
166,290398,NaN,NaN,NaN,NaN,NaN,NaN
167,295864,NaN,NaN,NaN,NaN,NaN,Different from clinician but I think it's fine
172,305128,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import pandas as pd
import numpy as np

physician_df     = Physician_label_100.copy()
new_baseline_df  = baseline_expanded.copy()
new_enhance_df   = enhance_expanded.copy()

LABEL_COLS = ["Domain", "Subdomain", "Error Code"]

def normalize(x):
    if pd.isna(x):
        return None
    return " ".join(str(x).split())

def add_label_tuple(df, mark_negatives=False):
    df = df.copy()

    # Normalize label columns (create if missing)
    for c in LABEL_COLS:
        if c in df.columns:
            df[c] = df[c].map(normalize)
        else:
            df[c] = None

    # Negative rows (all three are NA/None)
    is_negative = df[LABEL_COLS].isna().all(axis=1)

    # Build tuples for ALL rows, then null out negatives
    tuples = list(zip(df["Domain"], df["Subdomain"], df["Error Code"]))
    s = pd.Series(tuples, index=df.index, dtype=object)

    # For negatives, set to None
    s.loc[is_negative] = None

    df["label_tuple"] = s
    if mark_negatives:
        df["is_negative_old"] = is_negative

    return df

def group_label_sets(df):
    df = add_label_tuple(df, mark_negatives=False)
    # keep only positive rows with an actual tuple
    g = df.dropna(subset=["label_tuple"]).groupby("Index")["label_tuple"].apply(set)
    return g.to_dict()

# --- build sets from new pipelines ---
physician_df    = add_label_tuple(physician_df, mark_negatives=True)
baseline_sets   = group_label_sets(new_baseline_df)
enhanced_sets   = group_label_sets(new_enhance_df)

# --- captured logic you wanted ---
def captured_by(label_sets, row):
    # Old negative: captured if new pipeline has NO labels for this index
    if row["is_negative_old"]:
        return len(label_sets.get(row["Index"], set())) == 0
    # Old positive: captured if the exact tuple appears for this index
    return row["label_tuple"] in label_sets.get(row["Index"], set())

physician_df["Captured_by_new_baseline"] = physician_df.apply(lambda r: captured_by(baseline_sets, r), axis=1)
physician_df["Captured_by_new_enhanced"] = physician_df.apply(lambda r: captured_by(enhanced_sets, r), axis=1)
physician_df["Captured_by_new_either"]   = (
    physician_df["Captured_by_new_baseline"] | physician_df["Captured_by_new_enhanced"]
)

# Optional: explicit flag you asked for
physician_df["Old_case_type"] = np.where(physician_df["is_negative_old"], "negative", "positive")

# --- quick summaries ---
pos_mask = ~physician_df["is_negative_old"]
neg_mask =  physician_df["is_negative_old"]

pos_cov_baseline = physician_df.loc[pos_mask, "Captured_by_new_baseline"].mean() if pos_mask.any() else np.nan
pos_cov_enhanced = physician_df.loc[pos_mask, "Captured_by_new_enhanced"].mean() if pos_mask.any() else np.nan
pos_cov_either   = physician_df.loc[pos_mask, "Captured_by_new_either"].mean()   if pos_mask.any() else np.nan

neg_keep_baseline = physician_df.loc[neg_mask, "Captured_by_new_baseline"].mean() if neg_mask.any() else np.nan
neg_keep_enhanced = physician_df.loc[neg_mask, "Captured_by_new_enhanced"].mean() if neg_mask.any() else np.nan
neg_keep_either   = physician_df.loc[neg_mask, "Captured_by_new_either"].mean()   if neg_mask.any() else np.nan

print(f"[Positives] Coverage — Baseline: {pos_cov_baseline:.2%} | Enhanced: {pos_cov_enhanced:.2%} | Either: {pos_cov_either:.2%}")
print(f"[Negatives] Kept negative — Baseline: {neg_keep_baseline:.2%} | Enhanced: {neg_keep_enhanced:.2%} | Either: {neg_keep_either:.2%}")


[Positives] Coverage — Baseline: 59.68% | Enhanced: 66.94% | Either: 73.39%
[Negatives] Kept negative — Baseline: 14.71% | Enhanced: 17.65% | Either: 20.59%


In [10]:
physician_df

,Index,Source,Domain,Subdomain,Error Code,Rationale,Free Text Comments,label_tuple,is_negative_old,Captured_by_new_baseline,Captured_by_new_enhanced,Captured_by_new_either,Old_case_type
0,2432,Human,Clinical Reasoning,Workflow Recommendations,Violation of Standard Workflows,Clinic appears to schedule directly via messag...,NaN,"(Clinical Reasoning, Workflow Recommendations,...",False,False,False,False,positive
1,5598,Baseline,Clinical Reasoning,Comprehension of Patient Query,Off-topic or Irrelevant Reply to Patient Query,The response is irrelevant to the patient's or...,NaN,"(Clinical Reasoning, Comprehension of Patient ...",False,True,True,True,positive
2,5598,Enhanced,Clinical Reasoning,Comprehension of Patient Query,Off-topic or Irrelevant Reply to Patient Query,The response gives unrelated information about...,NaN,"(Clinical Reasoning, Comprehension of Patient ...",False,True,True,True,positive
3,11928,Human,Clinical Reasoning,Comprehension of Patient Context,Incorrect Clinical Information,LLM did not realize that the CBC had already b...,NaN,"(Clinical Reasoning, Comprehension of Patient ...",False,False,True,True,positive
4,11928,Enhanced,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,The instructions are contradictory since they ...,NaN,"(Communication Quality & Readability, Clarity,...",False,True,True,True,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,285732,Human,Clinical Reasoning,Assessment,Omitted Verification of Incomplete Patient Inf...,The LLM made a resopnse despite not being able...,NaN,"(Clinical Reasoning, Assessment, Omitted Verif...",False,True,True,True,positive
166,290398,NaN,None,None,None,NaN,NaN,None,True,False,False,False,negative
167,295864,NaN,None,None,None,NaN,Different from clinician but I think it's fine,None,True,False,False,False,negative
172,305128,NaN,None,None,None,NaN,NaN,None,True,True,True,True,negative


In [11]:
# --- Subanalysis by Source (positives only) ---

# filter physician positives that have a Source label
pos = physician_df[(~physician_df["is_negative_old"]) & (physician_df["Source"].notna())].copy()

# Label-level coverage by Source
label_summary = (
    pos.groupby("Source")
       .agg(
           n_labels=("Index", "size"),
           cov_baseline=("Captured_by_new_baseline", "mean"),
           cov_enhanced=("Captured_by_new_enhanced", "mean"),
           cov_either=("Captured_by_new_either", "mean"),
       )
       .reset_index()
)

# Convert proportions to %
for c in ["cov_baseline", "cov_enhanced", "cov_either"]:
    label_summary[c] = (label_summary[c] * 100).round(2)

print("\nLabel-level coverage by Source (%):")
print(label_summary.to_string(index=False))

# Index-level: “all labels at this (Source, Index) are covered”
index_summary = (
    pos.groupby(["Source", "Index"])
       .agg(
           all_baseline=("Captured_by_new_baseline", "all"),
           all_enhanced=("Captured_by_new_enhanced", "all"),
           all_either=("Captured_by_new_either", "all"),
       )
       .groupby("Source")
       .mean()
       .reset_index()
)

for c in ["all_baseline", "all_enhanced", "all_either"]:
    index_summary[c] = (index_summary[c] * 100).round(2)

print("\nIndex-level full coverage by Source (%):")
print(index_summary.to_string(index=False))



Label-level coverage by Source (%):
  Source  n_labels  cov_baseline  cov_enhanced  cov_either
Baseline        30         66.67         76.67       83.33
Enhanced        57         71.93         75.44       84.21
   Human        37         35.14         45.95       48.65

Index-level full coverage by Source (%):
  Source  all_baseline  all_enhanced  all_either
Baseline         55.56         66.67       72.22
Enhanced         54.55         50.00       63.64
   Human         35.29         44.12       47.06


In [12]:
physician_df

,Index,Source,Domain,Subdomain,Error Code,Rationale,Free Text Comments,label_tuple,is_negative_old,Captured_by_new_baseline,Captured_by_new_enhanced,Captured_by_new_either,Old_case_type
0,2432,Human,Clinical Reasoning,Workflow Recommendations,Violation of Standard Workflows,Clinic appears to schedule directly via messag...,NaN,"(Clinical Reasoning, Workflow Recommendations,...",False,False,False,False,positive
1,5598,Baseline,Clinical Reasoning,Comprehension of Patient Query,Off-topic or Irrelevant Reply to Patient Query,The response is irrelevant to the patient's or...,NaN,"(Clinical Reasoning, Comprehension of Patient ...",False,True,True,True,positive
2,5598,Enhanced,Clinical Reasoning,Comprehension of Patient Query,Off-topic or Irrelevant Reply to Patient Query,The response gives unrelated information about...,NaN,"(Clinical Reasoning, Comprehension of Patient ...",False,True,True,True,positive
3,11928,Human,Clinical Reasoning,Comprehension of Patient Context,Incorrect Clinical Information,LLM did not realize that the CBC had already b...,NaN,"(Clinical Reasoning, Comprehension of Patient ...",False,False,True,True,positive
4,11928,Enhanced,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,The instructions are contradictory since they ...,NaN,"(Communication Quality & Readability, Clarity,...",False,True,True,True,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,285732,Human,Clinical Reasoning,Assessment,Omitted Verification of Incomplete Patient Inf...,The LLM made a resopnse despite not being able...,NaN,"(Clinical Reasoning, Assessment, Omitted Verif...",False,True,True,True,positive
166,290398,NaN,None,None,None,NaN,NaN,None,True,False,False,False,negative
167,295864,NaN,None,None,None,NaN,Different from clinician but I think it's fine,None,True,False,False,False,negative
172,305128,NaN,None,None,None,NaN,NaN,None,True,True,True,True,negative


In [13]:
import pandas as pd
import numpy as np

# --- Reuse helpers from earlier (paste if not already in scope) ---
LABEL_COLS = ["Domain", "Subdomain", "Error Code"]

def normalize(x):
    if pd.isna(x):
        return None
    return " ".join(str(x).split())

def add_label_tuple(df, mark_negatives=False):
    df = df.copy()
    for c in LABEL_COLS:
        if c in df.columns:
            df[c] = df[c].map(normalize)
        else:
            df[c] = None
    is_negative = df[LABEL_COLS].isna().all(axis=1)
    tuples = list(zip(df["Domain"], df["Subdomain"], df["Error Code"]))
    s = pd.Series(tuples, index=df.index, dtype=object)
    s.loc[is_negative] = None
    df["label_tuple"] = s
    if mark_negatives:
        df["is_negative_old"] = is_negative
    return df

def group_label_sets(df):
    df = add_label_tuple(df, mark_negatives=False)
    df = df.dropna(subset=["label_tuple"])
    return df.groupby("Index")["label_tuple"].apply(set).to_dict()

# --- Ensure physician_df has tuples & build reference sets of old (positive) labels ---
physician_df = add_label_tuple(physician_df, mark_negatives=True)
physician_pos = physician_df[~physician_df["is_negative_old"]].copy()
physician_sets = group_label_sets(physician_pos)   # {Index: set of label_tuples}

# --- Function to tag new pipeline rows with membership in old physician labels ---
def annotate_exist_in_physician(df):
    df = add_label_tuple(df)  # builds df["label_tuple"] with same normalization
    def exists(row):
        return row["label_tuple"] in physician_sets.get(row["Index"], set())
    df["In_old_physician"] = df.apply(exists, axis=1)
    return df

# Apply to new pipelines
new_baseline_df = annotate_exist_in_physician(new_baseline_df)
new_enhance_df  = annotate_exist_in_physician(new_enhance_df)

# --- Quick sanity summaries (precision-ish) ---
prec_baseline = new_baseline_df["In_old_physician"].mean() if not new_baseline_df.empty else np.nan
prec_enhanced = new_enhance_df["In_old_physician"].mean()  if not new_enhance_df.empty else np.nan

print(f"[New Baseline] % of labels that match old physician: {prec_baseline:.2%}")
print(f"[New Enhanced] % of labels that match old physician: {prec_enhanced:.2%}")

# (Optional) per-index FP counts in the new pipelines
fp_counts_baseline = (
    new_baseline_df[~new_baseline_df["In_old_physician"]]
    .groupby("Index")
    .size()
    .rename("baseline_FP_count")
)
fp_counts_enhanced = (
    new_enhance_df[~new_enhance_df["In_old_physician"]]
    .groupby("Index")
    .size()
    .rename("enhanced_FP_count")
)

print("\nExample FP-heavy indices (top 10 baseline):")
print(fp_counts_baseline.sort_values(ascending=False).head(10).to_string())

print("\nExample FP-heavy indices (top 10 enhanced):")
print(fp_counts_enhanced.sort_values(ascending=False).head(10).to_string())


[New Baseline] % of labels that match old physician: 8.08%
[New Enhanced] % of labels that match old physician: 8.29%

Example FP-heavy indices (top 10 baseline):
Index
208055    17
135202    14
238215    14
89405     14
21392     13
72081     13
110120    13
214001    13
314021    13
241520    12

Example FP-heavy indices (top 10 enhanced):
Index
89405     19
44402     18
53391     15
135202    14
208055    14
21392     13
214001    13
104179    13
14858     13
262055    13


In [14]:
new_baseline_df

,Index,Domain,Subdomain,Error Code,Rationale,confidence,error_summary,error_highlights,subdomain_confidence,label_tuple,In_old_physician
0,2432,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,The directive to schedule only via the app wit...,0.68,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.70,"(Communication Quality & Readability, Clarity,...",False
1,2432,Clinical Reasoning,Comprehension of Patient Query,Misinterpretation of Clinical Query,Patient requested steps to see a colleague; re...,0.81,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.82,"(Clinical Reasoning, Comprehension of Patient ...",False
2,2432,Clinical Reasoning,Comprehension of Patient Query,Incomplete Response to Patient Query,It does not explain how to book with the colle...,0.79,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.82,"(Clinical Reasoning, Comprehension of Patient ...",False
3,2432,Clinical Reasoning,Workflow Recommendations,Omitted Workflow Information,Directs only to the app without offering phone...,0.87,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.86,"(Clinical Reasoning, Workflow Recommendations,...",False
4,5598,Clinical Reasoning,Comprehension of Patient Query,Incomplete Response to Patient Query,It does not acknowledge/confirm the appointmen...,0.87,The response does not acknowledge or confirm t...,"[{'id': 'H1', 'excerpt': 'I don't have access ...",0.87,"(Clinical Reasoning, Comprehension of Patient ...",False
...,...,...,...,...,...,...,...,...,...,...,...
725,314021,Clinical Reasoning,Triage,Inadequate Risk Stratification,The reply provides no assessment of risk level...,0.72,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.76,"(Clinical Reasoning, Triage, Inadequate Risk S...",False
726,314021,Clinical Reasoning,Workflow Recommendations,Omitted Workflow Information,No timeframe or next steps for referral placem...,0.78,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.78,"(Clinical Reasoning, Workflow Recommendations,...",False
727,314021,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,"Provides unclear directions (""place a referral...",0.79,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.79,"(Communication Quality & Readability, Clarity,...",False
728,314021,Communication Quality & Readability,Formatting,Formatting / Display Error,Message ends with an incomplete sign-off lacki...,0.66,The response acknowledges worsening behavior b...,"[{'id': 'H1', 'excerpt': 'Given the intensifie...",0.66,"(Communication Quality & Readability, Formatti...",False


In [15]:
# ----- Build physician positive and negative sets -----
pos_phys = physician_df[~physician_df[["Domain","Subdomain","Error Code"]].isna().all(axis=1)].copy()
neg_phys = physician_df[physician_df[["Domain","Subdomain","Error Code"]].isna().all(axis=1)].copy()

physician_sets = group_label_sets(pos_phys)
baseline_sets  = group_label_sets(new_baseline_df)
enhanced_sets  = group_label_sets(new_enhance_df)

def classify_with_TN(phys_sets, pipe_sets, pipeline_name):
    records = []

    # handle positive rows (TP, FN, FP)
    all_idx = set(phys_sets.keys()) | set(pipe_sets.keys())
    for idx in all_idx:
        old = phys_sets.get(idx, set())
        new = pipe_sets.get(idx, set())
        tp = old & new
        fn = old - new
        fp = new - old
        for (d, sd, ec) in tp:
            records.append((pipeline_name, idx, d, sd, ec, "TP"))
        for (d, sd, ec) in fn:
            records.append((pipeline_name, idx, d, sd, ec, "FN"))
        for (d, sd, ec) in fp:
            records.append((pipeline_name, idx, d, sd, ec, "FP"))

    # handle negative rows (TN vs FP)
    neg_indices = neg_phys["Index"].unique()
    for idx in neg_indices:
        has_new = idx in pipe_sets and len(pipe_sets[idx]) > 0
        if has_new:
            # pipeline predicted something where physician had none → FP(s)
            # we can mark once per neg row
            records.append((pipeline_name, idx, None, None, None, "FP"))
        else:
            records.append((pipeline_name, idx, None, None, None, "TN"))

    return pd.DataFrame(records, columns=["Pipeline","Index","Domain","Subdomain","Error Code","Status"])

# Build combined table
lbl_compare = pd.concat([
    classify_with_TN(physician_sets, baseline_sets, "Baseline"),
    classify_with_TN(physician_sets, enhanced_sets, "Enhanced"),
], ignore_index=True)

# ----- Confusion matrix -----
confusion = (
    lbl_compare.groupby(["Pipeline","Status"]).size()
    .unstack("Status", fill_value=0)[["TP","FN","FP","TN"]]
)

# Metrics
def metrics(tp, fn, fp, tn):
    prec = tp / (tp + fp) if (tp + fp) else 0.0
    rec  = tp / (tp + fn) if (tp + fn) else 0.0
    spec = tn / (tn + fp) if (tn + fp) else 0.0
    f1   = 2*prec*rec / (prec + rec) if (prec + rec) else 0.0
    return prec, rec, spec, f1

metrics_rows = []
for pipe, row in confusion.iterrows():
    p, r, s, f1 = metrics(row["TP"], row["FN"], row["FP"], row["TN"])
    metrics_rows.append((pipe, p, r, s, f1))
metrics_df = pd.DataFrame(metrics_rows, columns=["Pipeline","Precision","Recall","Specificity","F1"]).set_index("Pipeline")

print("Confusion matrix (label-level):")
print(confusion)
print("\nMetrics (label-level):")
print((metrics_df*100).round(2).astype(str) + "%")


Confusion matrix (label-level):
Status    TP  FN   FP  TN
Pipeline                 
Baseline  59  43  699   5
Enhanced  65  37  747   6

Metrics (label-level):
         Precision  Recall Specificity      F1
Pipeline                                      
Baseline     7.78%  57.84%       0.71%  13.72%
Enhanced      8.0%  63.73%        0.8%  14.22%


In [16]:
# ----- Breakdown by Domain, Subdomain, Error Code -----

# Domain-level breakdown
domain_conf = (
    lbl_compare.dropna(subset=["Domain"])
    .groupby(["Pipeline", "Domain", "Status"])
    .size()
    .unstack("Status", fill_value=0)
    .reset_index()
)

# Subdomain-level breakdown
subdomain_conf = (
    lbl_compare.dropna(subset=["Subdomain"])
    .groupby(["Pipeline", "Subdomain", "Status"])
    .size()
    .unstack("Status", fill_value=0)
    .reset_index()
)

# Error Code-level breakdown
ecode_conf = (
    lbl_compare.dropna(subset=["Error Code"])
    .groupby(["Pipeline", "Error Code", "Status"])
    .size()
    .unstack("Status", fill_value=0)
    .reset_index()
)

print("\nDomain-level confusion counts:")
print(domain_conf.head(20).to_string(index=False))

print("\nSubdomain-level confusion counts:")
print(subdomain_conf.head(20).to_string(index=False))

print("\nError Code-level confusion counts:")
print(ecode_conf.head(20).to_string(index=False))



Domain-level confusion counts:
Pipeline                              Domain  FN  FP  TP
Baseline                       Accessibility   1   0   0
Baseline               Bias & Stigmatization   0   8   2
Baseline                  Clinical Reasoning  37 504  42
Baseline Communication Quality & Readability   5 150  15
Baseline                  Privacy & Security   0   8   0
Enhanced                       Accessibility   1   0   0
Enhanced               Bias & Stigmatization   0   7   2
Enhanced                  Clinical Reasoning  34 551  45
Enhanced Communication Quality & Readability   2 158  18
Enhanced                  Privacy & Security   0   3   0

Subdomain-level confusion counts:
Pipeline                                                Subdomain  FN  FP  TP
Baseline                                               Assessment   3  53   4
Baseline                                                  Clarity   2  63   8
Baseline                                 Clinical Recommendations   5 13

In [17]:
baseline_expanded[baseline_expanded["Index"] == 2432].sort_values(by="Subdomain", ascending=False)

,Index,Domain,Subdomain,Error Code,Rationale,confidence,error_summary,error_highlights,subdomain_confidence
3,2432,Clinical Reasoning,Workflow Recommendations,Omitted Workflow Information,Directs only to the app without offering phone...,0.87,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.86
1,2432,Clinical Reasoning,Comprehension of Patient Query,Misinterpretation of Clinical Query,Patient requested steps to see a colleague; re...,0.81,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.82
2,2432,Clinical Reasoning,Comprehension of Patient Query,Incomplete Response to Patient Query,It does not explain how to book with the colle...,0.79,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.82
0,2432,Communication Quality & Readability,Clarity,Ambiguous or Conflicting Instructions,The directive to schedule only via the app wit...,0.68,The response is polite but has communication a...,"[{'id': 'H1', 'excerpt': 'Please use the Stanf...",0.70


In [18]:
physician_df[physician_df["Index"] == 2432]  

,Index,Source,Domain,Subdomain,Error Code,Rationale,Free Text Comments,label_tuple,is_negative_old,Captured_by_new_baseline,Captured_by_new_enhanced,Captured_by_new_either,Old_case_type
0,2432,Human,Clinical Reasoning,Workflow Recommendations,Violation of Standard Workflows,Clinic appears to schedule directly via messag...,NaN,"(Clinical Reasoning, Workflow Recommendations,...",False,False,False,False,positive
